In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, avg, max, min, hour

# Spark-Session erstellen
spark = SparkSession.builder.appName("Tankstellen Analyse").getOrCreate()

# Daten laden
preise = spark.read.csv("Aufgabe 1 - OnPremise-Engines/data/prices.csv", header=True, inferSchema=True)
stationen = spark.read.csv("Aufgabe 1 - OnPremise-Engines/data/stations.csv", header=True, inferSchema=True)

# 1. Tankstelle mit der stärksten Preisschwankung
preisschwankungen = preise.groupBy("station_id").agg(
    (max("price") - min("price")).alias("schwankung")
)
max_schwankung = preisschwankungen.orderBy(col("schwankung").desc()).first()
beste_tankstelle = stationen.filter(col("station_id") == max_schwankung["station_id"]).first()

print(f"Tankstelle mit der stärksten Schwankung: {beste_tankstelle['station_name']}")

# 2. Durchschnittspreis pro Tag
durchschnittspreis_pro_tag = preise.groupBy("station_id", "date").agg(
    avg("price").alias("durchschnittspreis")
)

durchschnittspreis_pro_tag.show()

# 3. Tageszeiten, die vermieden werden sollten
preise_mit_stunde = preise.withColumn("hour", hour(col("timestamp")))
durchschnittspreis_pro_stunde = preise_mit_stunde.groupBy("hour").agg(
    avg("price").alias("durchschnittspreis")
)

durchschnittspreis_pro_stunde.orderBy(col("durchschnittspreis").desc()).show()

# Ergebnisse speichern
preisschwankungen.write.csv("output/preisschwankungen", header=True)
durchschnittspreis_pro_tag.write.csv("output/durchschnittspreis_pro_tag", header=True)
durchschnittspreis_pro_stunde.write.csv("output/durchschnittspreis_pro_stunde", header=True)

# Spark-Session beenden
spark.stop()


AnalysisException: [PATH_NOT_FOUND] Path does not exist: file:/Users/daviddamico/Desktop/Schreibtisch – MacBook Pro von David D./5. Semester/BigDataPrograming2.0/Aufgabe 1 - OnPremise-Engines/src/Aufgabe 1 - OnPremise-Engines/data/prices.csv.